In [1]:
import sys
sys.path.append("D:/gnn-cloud-manufacturing/")

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import random
import openpyxl
import os
from cloudmanufacturing.data import create_distance_matrix
from cloudmanufacturing.data import create_excel_table
from cloudmanufacturing.data import solve_excel
from cloudmanufacturing.data import read_fatahi_dataset
from cloudmanufacturing.mip_solver import mip_solve
from cloudmanufacturing.validation import objvalue, construct_delta

c:\Users\ASUS Vivobook\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Генерация тренировочных данных

In [15]:
trios = []
for i in range(10):
    trios.append(tuple((random.randint(3,25),random.randint(3,25),random.randint(3,25))))
trios.sort()

In [16]:
file_path = '../data/train_data.xlsx'
wb = openpyxl.Workbook()
wb.save(file_path)

In [17]:
sheets = []
for n_operations, n_suboperations, n_cities in trios:
    sheets.append(create_excel_table(n_operations, n_suboperations, n_cities, n_problem=1, filepath=file_path ))

## Обучение на тренировочных данных

In [18]:
dataset = read_fatahi_dataset('../data/train_data.xlsx', sheets)

100%|██████████| 10/10 [00:02<00:00,  4.67it/s]


In [19]:
filepath = '../data/solved_gen_data.xlsx'
wb = openpyxl.Workbook()

wb.save(filepath)

In [21]:
filepath = './solved_gen_data.xlsx'
for problem in dataset:
    delta, gamma, status, value = mip_solve(problem)
    
    sheet_name = problem['name']
    city_to_op_solution = []
    otc=[]
    for o, t, c in zip(*np.where(gamma == 1)):
        city_to_op_solution.append((f'city {c}', f'{t}_{o}'))
        otc.append([o,t,c])
    shape_g = np.shape(gamma)
    solve_excel(fpath, sheet_name, city_to_op_solution, otc, shape_g)